In [1]:
'''CRF Implementation by Vinayak Chaturvedi using Hero Types'''

'CRF Implementation by Vinayak Chaturvedi using Hero Types'

In [2]:
'''Importing Basic Libraries'''
import pandas as pd
import numpy as np
import datetime


In [3]:
'''Following Cells will contain data loading and preprocessing steps'''
df = pd.read_csv("hero_ban_final.csv")

In [4]:

matrix1 =np.array(df.loc[:,['match_id', 'Result','index_h1','index_h2','index_h3','index_h4']])
matrix1

array([[1928898739, 1, '1/73', '2/69', '3/71', '4/7'],
       [1928932285, 1, '1/69', '2/102', '3/32', '4/61'],
       [1928903165, 1, '1/85', '2/68', '3/75', '4/2'],
       ...,
       [1829187140, 1, '1/44', '2/29', '3/12', '4/17'],
       [1829220503, -1, '1/1', '2/57', '3/50', '4/85'],
       [1829227757, 1, '1/71', '2/100', '3/7', '4/85']], dtype=object)

In [5]:
df1 = pd.read_csv("hero_names.csv")

In [20]:
matrix2 =np.array(df1.loc[:,['hero_id', 'Strength','Agility','Intelligence']])
matrix2

array([[  1,   0,   1,   0],
       [  2,   1,   0,   0],
       [  3,   0,   0,   1],
       [  4,   0,   1,   0],
       [  5,   0,   0,   1],
       [  6,   0,   1,   0],
       [  7,   1,   0,   0],
       [  8,   0,   1,   0],
       [  9,   0,   1,   0],
       [ 10,   0,   1,   0],
       [ 11,   0,   1,   0],
       [ 12,   0,   1,   0],
       [ 13,   0,   0,   1],
       [ 14,   1,   0,   0],
       [ 15,   0,   1,   0],
       [ 16,   1,   0,   0],
       [ 17,   0,   0,   1],
       [ 18,   1,   0,   0],
       [ 19,   1,   0,   0],
       [ 20,   0,   1,   1],
       [ 21,   0,   0,   1],
       [ 22,   0,   0,   1],
       [ 23,   1,   0,   0],
       [ 24,   1,   1,   1],
       [ 25,   0,   0,   1],
       [ 26,   0,   0,   1],
       [ 27,   0,   0,   1],
       [ 28,   1,   0,   0],
       [ 29,   1,   0,   0],
       [ 30,   0,   0,   1],
       [ 31,   0,   0,   1],
       [ 32,   0,   1,   0],
       [ 33,   0,   0,   1],
       [ 34,   0,   0,   1],
       [ 35,  

In [7]:
'''Importing libraries for CRF implementation'''
from itertools import chain

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve, GridSearchCV

In [27]:
'''Tagger function to prepare dataset to feed into CRF'''
def tagging(match, hero_matrix):
    final_list=[]
    middle_value=""
    for element in range(2,5):
        #print("Match is: ",match[element][1])
        middle_tag = ""
        hero_entry = 0
        index_value = 0
        #length = len(match[element])
        temp_list = hero_matrix[int(match[element][2:])-1]
        if temp_list[1]==1:
            middle_tag = "St"
        elif temp_list[2]==1:
            middle_tag = "Ag"
        elif temp_list[3]==1:
            middle_tag = "It"
        if element==5:
            index_value = 1000 
        else:
            index_value = int(match[element+1][2:])
            hero_num = hero_matrix[index_value-1]
            if hero_num[1]==1:
                index_value = 1
            elif hero_num[2]==1:
                index_value = 2
            elif hero_num[3]==1:
                index_value = 3
            
        hero_entry = match[element][2:]
        middle_value +=  middle_tag + "/"
        final_list.append((hero_entry, middle_value, index_value))
             
    return final_list
        

dataset = []
for i in range(len(matrix1)):
    dataset.append(tagging(matrix1[i], matrix2))

train_sent = dataset[0:40000]
test_sent = dataset[40000:50000]
train_sent

[[('73', 'St/', 1), ('69', 'St/St/', 1), ('71', 'St/St/St/', 1)],
 [('69', 'St/', 1), ('102', 'St/St/', 2), ('32', 'St/St/Ag/', 2)],
 [('85', 'St/', 3), ('68', 'St/It/', 3), ('75', 'St/It/It/', 1)],
 [('71', 'St/', 1), ('73', 'St/St/', 1), ('85', 'St/St/St/', 3)],
 [('12', 'Ag/', 1), ('42', 'Ag/St/', 1), ('7', 'Ag/St/St/', 2)],
 [('75', 'It/', 1), ('85', 'It/St/', 2), ('41', 'It/St/Ag/', 1)],
 [('85', 'St/', 3), ('75', 'St/It/', 1), ('69', 'St/It/St/', 1)],
 [('59', 'St/', 3), ('75', 'St/It/', 3), ('5', 'St/It/It/', 3)],
 [('75', 'It/', 3), ('57', 'It/It/', 2), ('41', 'It/It/Ag/', 1)],
 [('69', 'St/', 1), ('85', 'St/St/', 3), ('112', 'St/St/It/', 1)],
 [('73', 'St/', 2), ('1', 'St/Ag/', 1), ('69', 'St/Ag/St/', 2)],
 [('32', 'Ag/', 3), ('21', 'Ag/It/', 1), ('29', 'Ag/It/St/', 3)],
 [('75', 'It/', 3), ('57', 'It/It/', 1), ('71', 'It/It/St/', 2)],
 [('75', 'It/', 3), ('33', 'It/It/', 2), ('1', 'It/It/Ag/', 3)],
 [('69', 'St/', 1), ('91', 'St/St/', 1), ('100', 'St/St/St/', 2)],
 [('100', '

In [44]:
'''Feature Set 1. Please use this cell !'''
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    #print("feature",word, postag, i)
    #print(postag[0:2])
    features = {
        'bias': 1.0,
        'first_2_heros_int': (i==1 and postag[0:2]=='It' and postag[3:5]=='It'),
        'first_2_heros_st': (i==1 and postag[0:2]=='It' and postag[3:5]=='St'),
        'first_2_heros_ag': (i==1 and postag[0:2]=='It' and postag[3:5]=='Ag'),
        'alternate_condition_itst': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='St')or(postag[0:2]=='St' and postag[3:5]=='It'))),
        'alternate_condition_stag': (i==1 and ((postag[0:2]=='St' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='St'))),
        'alternate_condition_itag': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='It'))),
        
    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [30]:
'''Feature set 2. I have not used this one please proceed further without running this cell!!'''
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = {
        'bias': 1.0,
        'first_2_heros_int': (i==1 and postag[0:2]=='It' and postag[3:5]=='It'),
        'first_2_heros_st': (i==1 and postag[0:2]=='It' and postag[3:5]=='St'),
        'first_2_heros_ag': (i==1 and postag[0:2]=='It' and postag[3:5]=='Ag'),
        'alternate_condition_itst': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='St')or(postag[0:2]=='St' and postag[3:5]=='It'))),
        'alternate_condition_stag': (i==1 and ((postag[0:2]=='St' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='St'))),
        'alternate_condition_itag': (i==1 and ((postag[0:2]=='It' and postag[3:5]=='Ag')or(postag[0:2]=='Ag' and postag[3:5]=='It'))),
        'third_hero_st': (i==2 and postag[6:8]=='St'),
        'third_hero_it': (i==2 and postag[6:8]=='It'),
        'third_hero_ag': (i==2 and postag[6:8]=='Ag'),
        'second_pick_int': (i==1 and postag[3:5]=='It'),
        'second_pick_ag': (i==1 and postag[3:5]=='Ag'),
        'second_pick_st': (i==1 and postag[3:5]=='St'),
        'first_pick_int': (i==0 and postag[0:2]=='It'),
        'first_pick_st': (i==0 and postag[0:2]=='St'),
        'first_pick_ag': (i==0 and postag[0:2]=='Ag'),
        
    }

    return features


    
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [str(label) for token, postag, label in sent]

In [45]:
'''Preparation of train and text data'''
sent2features(train_sent[0])
#sent2labels(train_sent[0])
X_train = [sent2features(s) for s in train_sent]
y_train = [sent2labels(s) for s in train_sent]
X_test = [sent2features(s) for s in test_sent]
y_test = [sent2labels(s) for s in test_sent]
len(X_train)
X_train
#y_train

[[{'bias': 1.0,
   'first_2_heros_int': False,
   'first_2_heros_st': False,
   'first_2_heros_ag': False,
   'alternate_condition_itst': False,
   'alternate_condition_stag': False,
   'alternate_condition_itag': False},
  {'bias': 1.0,
   'first_2_heros_int': False,
   'first_2_heros_st': False,
   'first_2_heros_ag': False,
   'alternate_condition_itst': False,
   'alternate_condition_stag': False,
   'alternate_condition_itag': False},
  {'bias': 1.0,
   'first_2_heros_int': False,
   'first_2_heros_st': False,
   'first_2_heros_ag': False,
   'alternate_condition_itst': False,
   'alternate_condition_stag': False,
   'alternate_condition_itag': False}],
 [{'bias': 1.0,
   'first_2_heros_int': False,
   'first_2_heros_st': False,
   'first_2_heros_ag': False,
   'alternate_condition_itst': False,
   'alternate_condition_stag': False,
   'alternate_condition_itag': False},
  {'bias': 1.0,
   'first_2_heros_int': False,
   'first_2_heros_st': False,
   'first_2_heros_ag': False,
   '

In [46]:
'''CRF suite is used to train the model'''
crf = sklearn_crfsuite.CRF(
    algorithm='l2sgd',
    #c1=0.1,
    c2=1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(algorithm='l2sgd', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=None, c2=1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [59]:
'''3 Labels according to types strength,agility and intelligence respectively
'''
my_labels=['1', '2', '3']
X_test= np.array(X_test)
X_test.shape


(10000, 3)

In [ ]:
y_pred = crf.predict(X_test)
f1_val = metrics.flat_f1_score(y_test, y_pred,
                      average='macro', labels=my_labels)
acc_val = sklearn_crfsuite.metrics.flat_accuracy_score(y_test, y_pred)
precision_val = metrics.flat_precision_score(y_test, y_pred,average='macro',labels=my_labels)
recall_val = metrics.flat_recall_score(y_test, y_pred,average='macro',labels=my_labels)

print("F1 value is: ",f1_val)
print("Accuracy value is: ",acc_val)
print("Precision is: ",precision_val)
print("Recall is: ",recall_val)

In [62]:
'''Let us make predictions.The accuracy here is a true representation. I am going to measure accuracies seperately as well for verification'''
import random as ra
for i in range(0,7):
    print("Iteration Number : ",i)
    a = ra.randint(0,9999)
    b = ra.randint(0,9999)
    while (a>=b) and (b-a<500):
        a = ra.randint(0,9999)
        b = ra.randint(0,9999)
    y_pred = crf.predict(X_test[a:b,:])
    f1_val = metrics.flat_f1_score(y_test[a:b], y_pred,
                          average='macro', labels=my_labels)
    acc_val = sklearn_crfsuite.metrics.flat_accuracy_score(y_test[a:b], y_pred)
    precision_val = metrics.flat_precision_score(y_test[a:b], y_pred,average='macro',labels=my_labels)
    recall_val = metrics.flat_recall_score(y_test[a:b], y_pred,average='macro',labels=my_labels)

    print("F1 value is: ",f1_val)
    print("Accuracy value is: ",acc_val)
    print("Precision is: ",precision_val)
    print("Recall is: ",recall_val)

Iteration Number :  0
F1 value is:  0.20736359806712548
Accuracy value is:  0.3995726495726496
Precision is:  0.2552955530088336
Recall is:  0.33414821205865824
Iteration Number :  1
F1 value is:  0.20457261138419458
Accuracy value is:  0.39200483888705595
Precision is:  0.25045802931191286
Recall is:  0.33402522974427457
Iteration Number :  2
F1 value is:  0.20314630559177815
Accuracy value is:  0.3837084673097535
Precision is:  0.2298989898989899
Recall is:  0.33434226380937554
Iteration Number :  3
F1 value is:  0.20710104455448053
Accuracy value is:  0.39407651172357056
Precision is:  0.2556546429787117
Recall is:  0.3349821603514493
Iteration Number :  4
F1 value is:  0.20664444256229567
Accuracy value is:  0.397087994921844
Precision is:  0.2543113986514381
Recall is:  0.3341110314883015
Iteration Number :  5
F1 value is:  0.21036299216326662
Accuracy value is:  0.40625377278763736
Precision is:  0.2593813460980563
Recall is:  0.33381096011526984
Iteration Number :  6
F1 value is

In [56]:
import random as ra
my_ytest = np.array(y_test)
my_ypred = np.array(y_pred)
accuracy_test=0
for j in range (0,7):
    a = ra.randint(0,9999)
    b = ra.randint(0,9999)
    while (a>=b) and (b-a<500):
        a = ra.randint(0,9999)
        b = ra.randint(0,9999)
    for i in range(0,3):

        print("Printing for hero :",i+1)
        print("------------------------------------------------------")
        my_ytest_hero = my_ytest[a:b,i]
        my_ypred_hero = my_ypred[a:b,i]
        divisor = my_ytest.shape[0]
        sum_of_correct_predictions = sum(p == t for p, t in zip(my_ypred_hero, my_ytest_hero))
        my_accuracy = (sum_of_correct_predictions/divisor)*100
        accuracy_test=accuracy_test+my_accuracy
        print("Accuracy for hero number",i+1," is:",my_accuracy)
    total_accuracy = accuracy_test/3
print("Total Accuracy is :",total_accuracy)

Printing for hero : 1
------------------------------------------------------
Accuracy for hero number 1  is: 10.85
Printing for hero : 2
------------------------------------------------------
Accuracy for hero number 2  is: 9.16
Printing for hero : 3
------------------------------------------------------
Accuracy for hero number 3  is: 9.39
Printing for hero : 1
------------------------------------------------------
Accuracy for hero number 1  is: 14.549999999999999
Printing for hero : 2
------------------------------------------------------
Accuracy for hero number 2  is: 12.55
Printing for hero : 3
------------------------------------------------------
Accuracy for hero number 3  is: 12.82
Printing for hero : 1
------------------------------------------------------
Accuracy for hero number 1  is: 0.44999999999999996
Printing for hero : 2
------------------------------------------------------
Accuracy for hero number 2  is: 0.33
Printing for hero : 3
----------------------------------